In [21]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [22]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['NAME', 'EIN'])

In [23]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts()<1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [24]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [45]:
# ASK_AMT could use some cleaning
application_df['ASK_AMT'].value_counts()

ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64

In [49]:
# Choose a cutoff value and create a list of amounts of money requested to be replaced
ask_amt_to_replace = list(application_df['ASK_AMT'].value_counts()[application_df['ASK_AMT'].value_counts()<1000].index)

# Replace in dataframe
for aamt in ask_amt_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(aamt,"Over_5k")

# Check to make sure replacement was successful
application_df['ASK_AMT'].value_counts()

ASK_AMT
5000       25398
Over_5k     8901
Name: count, dtype: int64

In [52]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_application_df = pd.get_dummies(application_df, dtype = float)
dummy_application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_Over_5k
0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [53]:
# Split our preprocessed data into our features and target arrays
y = dummy_application_df['IS_SUCCESSFUL']
X = dummy_application_df.drop(columns = 'IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [54]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [58]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=41))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 15)                630       
                                                                 
 dense_4 (Dense)             (None, 15)                240       
                                                                 
 dense_5 (Dense)             (None, 1)                 16        
                                                                 
Total params: 886 (3.46 KB)
Trainable params: 886 (3.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5897 - accuracy: 0.7075
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5601 - accuracy: 0.7264
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5560 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7311
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7324
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5510 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7337
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7317
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5501 - accura

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5577 - accuracy: 0.7282 - 453ms/epoch - 2ms/step
Loss: 0.5576982498168945, Accuracy: 0.7281632423400879


In [61]:
#Just adjusting the ASK_AMT was not the answer, this version is slightly less accurate. Maybe adding another layer will help?
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=41))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=15, activation="relu"))

#Third hidden layer!
nn2.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 15)                630       
                                                                 
 dense_7 (Dense)             (None, 15)                240       
                                                                 
 dense_8 (Dense)             (None, 15)                240       
                                                                 
 dense_9 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1126 (4.40 KB)
Trainable params: 1126 (4.40 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5418 - accuracy: 0.7367
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accuracy: 0.7375
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accuracy: 0.7373
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accuracy: 0.7374
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5418 - accuracy: 0.7384
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5413 - accuracy: 0.7376
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5419 - accuracy: 0.7365
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5414 - accuracy: 0.7376
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5417 - accuracy: 0.7371
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5415 - accura

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7498 - accuracy: 0.5291 - 1s/epoch - 4ms/step
Loss: 0.7498183250427246, Accuracy: 0.5290961861610413


In [66]:
#Oh so that made it worse. Okay, back to the two layers... Maybe more neurons will help?
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=25, activation="relu", input_dim=41))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 25)                1050      
                                                                 
 dense_11 (Dense)            (None, 25)                650       
                                                                 
 dense_12 (Dense)            (None, 1)                 26        
                                                                 
Total params: 1726 (6.74 KB)
Trainable params: 1726 (6.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [68]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5863 - accuracy: 0.7112
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5599 - accuracy: 0.7293
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7287
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5532 - accuracy: 0.7298
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5535 - accuracy: 0.7290
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7305
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5498 - accuracy: 0.7311
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accura

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5559 - accuracy: 0.7297 - 857ms/epoch - 3ms/step
Loss: 0.5558791160583496, Accuracy: 0.72967928647995


In [70]:
#That looks like the right track, although not by a whole lot. I will add a few more neurons and give a new activation a shot.

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=30, activation="leaky_relu", input_dim=41))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=30, activation="leaky_relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 30)                1260      
                                                                 
 dense_14 (Dense)            (None, 30)                930       
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 2221 (8.68 KB)
Trainable params: 2221 (8.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [72]:
# Train the model
fit_model = nn4.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5777 - accuracy: 0.7154
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5603 - accuracy: 0.7272
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5571 - accuracy: 0.7280
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7302
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7291
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5532 - accuracy: 0.7291
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5528 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7305
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5509 - accuracy: 0.7289
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5507 - accura

In [73]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5566 - accuracy: 0.7300 - 1s/epoch - 4ms/step
Loss: 0.5566003322601318, Accuracy: 0.7300291657447815


In [74]:
#.0004 accuracy gain is probably not very helpful. One more shot with another activation, since adding neurons to make up the difference to 0.75 seems impractical.

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(tf.keras.layers.Dense(units=30, activation="tanh", input_dim=41))

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Output layer
nn5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 30)                1260      
                                                                 
 dense_17 (Dense)            (None, 30)                930       
                                                                 
 dense_18 (Dense)            (None, 1)                 31        
                                                                 
Total params: 2221 (8.68 KB)
Trainable params: 2221 (8.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
# Compile the model
nn5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [76]:
# Train the model
fit_model = nn5.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5824 - accuracy: 0.7107
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5607 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5561 - accuracy: 0.7291
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7293
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7302
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5515 - accuracy: 0.7306
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7310
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7326
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7318
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5492 - accura

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5583 - accuracy: 0.7290 - 878ms/epoch - 3ms/step
Loss: 0.5583471059799194, Accuracy: 0.7289795875549316


In [78]:
#And that was worse. I definitely see the use of kerastuner, trying to find the perfect combination manually is not an efficient or effective practice.

In [79]:
# Export our model to HDF5 file
#Exporting nn4 as that was the high point for accuracy
from google.colab import files

nn4.save('/content/AlphabetSoupCharity_Optimization.h5')
files.download('/content/AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>